# 循环神经网络

*循环神经网络*（recurrent neural networks，RNNs）
是具有隐状态的神经网络。
在介绍循环神经网络模型之前，
我们首先回顾 :numref:`sec_mlp`中介绍的多层感知机模型。

In [1]:
import torch
from d2l import torch as d2l

我们刚才提到，隐状态中
$\mathbf{X}_t \mathbf{W}_{xh} + \mathbf{H}_{t-1} \mathbf{W}_{hh}$的计算，
相当于$\mathbf{X}_t$和$\mathbf{H}_{t-1}$的拼接
与$\mathbf{W}_{xh}$和$\mathbf{W}_{hh}$的拼接的矩阵乘法。
虽然这个性质可以通过数学证明，
但在下面我们使用一个简单的代码来说明一下。
首先，我们定义矩阵`X`、`W_xh`、`H`和`W_hh`，
它们的形状分别为$(3，1)$、$(1，4)$、$(3，4)$和$(4，4)$。
分别将`X`乘以`W_xh`，将`H`乘以`W_hh`，
然后将这两个乘法相加，我们得到一个形状为$(3，4)$的矩阵。

In [35]:
# 输入 (X + W_xh) + (H + W_hh)
X    = torch.normal(0, 1, (3, 1))
W_xh = torch.normal(0, 1, (1, 4))


H    = torch.normal(0, 1, (3, 4))
W_hh = torch.normal(0, 1, (4, 4))

print(X)
print(W_xh)

tensor([[-0.0408],
        [-1.7170],
        [-0.2907]])
tensor([[-0.0349,  1.0426, -0.0222,  0.8779]])


In [36]:
print(H)
print(W_hh)

tensor([[-0.0308,  2.1261, -0.0737, -0.7566],
        [-0.7136,  0.6886,  1.0068, -1.2115],
        [ 0.5865,  0.8969,  0.1104, -1.7619]])
tensor([[ 0.8581, -0.5493,  0.8426,  0.2695],
        [ 0.9408, -0.1433,  1.1685,  1.3606],
        [-1.6592, -0.6335, -0.4893, -1.3101],
        [-0.4059, -0.2835, -0.2765,  0.5845]])


In [37]:
# 矩阵拼接
torch.matmul(X, W_xh) + torch.matmul(H, W_hh) 

tensor([[ 2.4047, -0.0692,  2.7045,  2.5028],
        [-1.0832, -1.7912,  0.0837, -2.7898],
        [ 1.8893, -0.3242,  1.9818, -0.0512]])

现在，我们沿列（轴1）拼接矩阵`X`和`H`，
沿行（轴0）拼接矩阵`W_xh`和`W_hh`。
这两个拼接分别产生形状$(3, 5)$和形状$(5, 4)$的矩阵。
再将这两个拼接的矩阵相乘，
我们得到与上面相同形状$(3, 4)$的输出矩阵。


In [42]:
print(X)
print(H)
print("拼接: ")
res1 = torch.cat((X, H), 1)

tensor([[-0.0408],
        [-1.7170],
        [-0.2907]])
tensor([[-0.0308,  2.1261, -0.0737, -0.7566],
        [-0.7136,  0.6886,  1.0068, -1.2115],
        [ 0.5865,  0.8969,  0.1104, -1.7619]])
拼接: 


In [43]:
print(W_xh)
print(W_hh)
print("拼接: ")
res2 = torch.cat((W_xh, W_hh), 0)

tensor([[-0.0349,  1.0426, -0.0222,  0.8779]])
tensor([[ 0.8581, -0.5493,  0.8426,  0.2695],
        [ 0.9408, -0.1433,  1.1685,  1.3606],
        [-1.6592, -0.6335, -0.4893, -1.3101],
        [-0.4059, -0.2835, -0.2765,  0.5845]])
拼接: 


In [44]:
# 矩阵相乘
torch.matmul(res1, res2)

tensor([[ 2.4047, -0.0692,  2.7045,  2.5028],
        [-1.0832, -1.7912,  0.0837, -2.7898],
        [ 1.8893, -0.3242,  1.9818, -0.0512]])